In [1]:
# Lab 9 XOR
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [4]:
# nn layers_MLP(멀티 레이어 퍼셉트론)
# layer 한 개(linear2)가 더 추가된 점이 Lab-08_1과 다른 점
# xor-nn
linear1 = torch.nn.Linear(2, 2, bias=True)
linear2 = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

In [5]:
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

In [6]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

In [7]:
for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

0 0.7434073090553284
100 0.693165123462677
200 0.6931577920913696
300 0.6931517124176025
400 0.6931463479995728
500 0.6931411027908325
600 0.6931357383728027
700 0.6931295394897461
800 0.6931220889091492
900 0.6931126117706299
1000 0.6930999755859375
1100 0.693082332611084
1200 0.6930569410324097
1300 0.6930190324783325
1400 0.6929606199264526
1500 0.6928660273551941
1600 0.6927032470703125
1700 0.6923960447311401
1800 0.6917302012443542
1900 0.6899654269218445
2000 0.6838316321372986
2100 0.6561669707298279
2200 0.4311031997203827
2300 0.1348937451839447
2400 0.06630446761846542
2500 0.04216821864247322
2600 0.030453890562057495
2700 0.02366594970226288
2800 0.019277770072221756
2900 0.016224045306444168
3000 0.013983819633722305
3100 0.012273941189050674
3200 0.01092812605202198
3300 0.00984247401356697
3400 0.008949029259383678
3500 0.008201331831514835
3600 0.007566758431494236
3700 0.007021680474281311
3800 0.006548587698489428
3900 0.0061342488043010235
4000 0.005768368486315012


In [8]:
# Accuracy 계산
# True if hypothesis > 0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), 
          '\nCorrect: ', predicted.detach().cpu().numpy(), 
          '\nAccuracy: ', accuracy.item())


Hypothesis:  [[0.00106364]
 [0.99889404]
 [0.99889404]
 [0.00165861]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


linear layer를 총 4개 사용하고 (deeper),
hidden layers의 weight을 10씩 사용(wider)하자.

이 케이스에서는 cost가 위 케이스에 비해 더 빨리 줄어든다

In [9]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

# xor-nn-wide-deep
linear1 = torch.nn.Linear(2, 10, bias=True)
linear2 = torch.nn.Linear(10, 10, bias=True)
linear3 = torch.nn.Linear(10, 10, bias=True)
linear4 = torch.nn.Linear(10, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid,
                            linear3, sigmoid, linear4, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

0 0.6978083848953247
100 0.693110466003418
200 0.6931020021438599
300 0.6930921673774719
400 0.6930808424949646
500 0.6930676698684692
600 0.6930519342422485
700 0.6930330395698547
800 0.6930099129676819
900 0.6929808855056763
1000 0.6929439902305603
1100 0.692895770072937
1200 0.6928312182426453
1300 0.6927415132522583
1400 0.6926121115684509
1500 0.692415714263916
1600 0.6920979022979736
1700 0.6915386915206909
1800 0.6904336214065552
1900 0.687838077545166
2000 0.6797102689743042
2100 0.640831708908081
2200 0.5488755702972412
2300 0.5083183646202087
2400 0.48752063512802124
2500 0.3293827176094055
2600 0.020208630710840225
2700 0.007720681838691235
2800 0.004509110003709793
2900 0.003107273718342185
3000 0.002338963560760021
3100 0.0018599143950268626
3200 0.0015352165792137384
3300 0.001301888725720346
3400 0.0011267683003097773
3500 0.000990953529253602
3600 0.0008827609708532691
3700 0.0007946972036734223
3800 0.0007217694655992091
3900 0.0006604068912565708
4000 0.00060818542260

In [10]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), 
          '\nCorrect: ', predicted.detach().cpu().numpy(), 
          '\nAccuracy: ', accuracy.item())


Hypothesis:  [[7.4389551e-05]
 [9.9988055e-01]
 [9.9990404e-01]
 [8.8267479e-05]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


### Backpropagation

여기서부터는 유튭 강의에만 나오는 내용으로,

오차역전파를 step-by-step 구현해보는 코드이다:

In [11]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [12]:
# nn Layers: nn.Linear() 2개를 사용
w1 = torch.Tensor(2, 2).to(device)
b1 = torch.Tensor(2).to(device)
w2 = torch.Tensor(2, 1).to(device)
b2 = torch.Tensor(1).to(device)

In [13]:
def sigmoid(x):
    return 1.0 / (1.0 + torch.exp(-x))
    # return torch.div(torch.tensor(1), torch.add(torch.tensor(1.0), torch.exp(-x)))

# sigmoid 함수의 미분
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [14]:
learning_rate = 0.01

In [15]:
for step in range(10001):
    # forward
    l1 = torch.add(torch.matmul(X, w1), b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1, w2), b2)
    Y_pred = sigmoid(l2)

    # BCE loss
    cost = -torch.mean(Y * torch.log(Y_pred) + 
                       (1 - Y) * torch.log(1 - Y_pred))
    
    # back propagation(chain rule)
    # Loss derivative
    d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred) + 1e-7)

    # Layer 2
    d_l2 = d_Y_pred * sigmoid_prime(l2)
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_b2)

    # Layer 1
    d_a1 = torch.matmul(d_b2, torch.transpose(w2, 0, 1))
    d_l1 = d_a1 * sigmoid_prime(l1)
    d_b1 =d_l1
    d_w1 = torch.matmul(torch.transpose(X, 0, 1), d_b1)

    # weight 업데이트
    w1 = w1 - learning_rate * d_w1
    b1 = b1 - learning_rate * torch.mean(d_b1, 0)
    w2 = w2 - learning_rate * d_w2
    b2 = b2 - learning_rate * torch.mean(d_b2, 0)

    if step % 100 == 0:
        print(step, cost.item())


0 2.0181491374969482
100 0.9342283010482788
200 0.7055551409721375
300 0.694060206413269
400 0.6935915946960449
500 0.6935281157493591
600 0.6934850215911865
700 0.6934477686882019
800 0.6934151649475098
900 0.6933867335319519
1000 0.6933616399765015
1100 0.6933394074440002
1200 0.6933199763298035
1300 0.6933026313781738
1400 0.6932873725891113
1500 0.6932736039161682
1600 0.693261444568634
1700 0.6932506561279297
1800 0.6932408213615417
1900 0.6932320594787598
2000 0.693224310874939
2100 0.6932173371315002
2200 0.693211019039154
2300 0.6932052969932556
2400 0.6932002305984497
2500 0.6931954622268677
2600 0.6931912899017334
2700 0.6931875944137573
2800 0.6931840777397156
2900 0.6931809186935425
3000 0.693178117275238
3100 0.6931754946708679
3200 0.6931731700897217
3300 0.6931710243225098
3400 0.693169116973877
3500 0.6931672692298889
3600 0.69316565990448
3700 0.6931642293930054
3800 0.693162739276886
3900 0.6931615471839905
4000 0.6931604146957397
4100 0.6931593418121338
4200 0.693158